In [ ]:
%matplotlib widget

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from ipywidgets import interactive, FloatSlider, IntSlider, VBox, HBox
from IPython.display import HTML, display
 
class Node:
    def __init__(self, id, threshold=0.5):
        self.id = id
        self.state = 0  # 0 = inactive, 1 = active
        self.threshold = threshold
        self.input_signal = 0
        self.refractory_timer = 0  # Counter for refractory period

    def update_state(self):
        if self.refractory_timer > 0:  # If in refractory period
            self.state = 0
            self.refractory_timer -= 1
        elif self.input_signal > self.threshold:
            self.state = 1
            self.refractory_timer = 3  # Set refractory period (3 steps)
        else:
            self.state = 0
        self.input_signal = 0  # Reset signal after processing


In [ ]:


class DynamicNetwork:
    def __init__(self, num_nodes, emotional_centers=2):
        self.graph = nx.watts_strogatz_graph(n=num_nodes, k=4, p=0.2)
        for node in self.graph.nodes:
            self.graph.nodes[node]['node'] = Node(node, threshold=np.random.uniform(0.3, 0.7))
        for edge in self.graph.edges:
            self.graph.edges[edge]['weight'] = np.random.uniform(0.1, 1.0)
        self.stimulus_node = np.random.randint(0, num_nodes)
        self.emotional_centers = np.random.choice(num_nodes, size=emotional_centers, replace=False)

    def propagate_signal(self, stimulus_prob=0.1):
        if np.random.rand() < stimulus_prob:
            self.graph.nodes[self.stimulus_node]['node'].state = 1

        for edge in self.graph.edges(data=True):
            source, target = edge[0], edge[1]
            signal = self.graph.nodes[source]['node'].state * edge[2]['weight']
            self.graph.nodes[target]['node'].input_signal += signal

        for center in self.emotional_centers:
            for neighbor in self.graph.neighbors(center):
                if self.graph.nodes[center]['node'].state == 1:
                    self.graph.nodes[neighbor]['node'].input_signal += 0.5

        for node_id in self.graph.nodes:
            self.graph.nodes[node_id]['node'].update_state()

    def get_active_nodes(self):
        active_nodes = [
            node_id
            for node_id, node_data in self.graph.nodes(data=True)
            if node_data['node'].state == 1
        ]
        return active_nodes


In [ ]:
# Initialize the network with a specific number of nodes
num_nodes = 10  # You can adjust this as needed
network = DynamicNetwork(num_nodes=num_nodes)


In [ ]:
def draw_emotional_network(graph, ax):
    pos = nx.spring_layout(graph)
    ax.clear()
    for edge in graph.edges:
        x = [pos[edge[0]][0], pos[edge[1]][0]]
        y = [pos[edge[0]][1], pos[edge[1]][1]]
        ax.plot(x, y, 'k-', alpha=0.7)

    node_colors = []
    for node in graph.nodes:
        if node in network.emotional_centers:
            node_colors.append('yellow')
        elif graph.nodes[node]['node'].state == 1:
            node_colors.append('red')
        else:
            node_colors.append('blue')
    nx.draw_networkx_nodes(graph, pos, ax=ax, node_color=node_colors, node_size=100, alpha=0.9)
    nx.draw_networkx_labels(graph, pos, ax=ax, font_size=8)
    ax.set_title('Emotional Neural Network')


In [ ]:
def animate_network(network, steps=20):
    fig, ax = plt.subplots(figsize=(8, 8))

    def update(frame):
        network.propagate_signal(stimulus_prob=stimulus_slider.value)
        draw_emotional_network(network.graph, ax)
        ax.set_title(f"Step {frame + 1}")

    ani = FuncAnimation(fig, update, frames=steps_slider.value, repeat=False)
    plt.close(fig)
    return ani

from ipywidgets import interact, IntSlider, FloatSlider

# Create sliders for adjusting parameters
node_slider = IntSlider(value=10, min=5, max=100, step=1, description='Nodes:')
threshold_slider = FloatSlider(value=0.5, min=0.1, max=1.0, step=0.05, description='Threshold:')

def update_network(nodes, threshold):
    global network
    network = DynamicNetwork(num_nodes=nodes)
    for node_data in network.graph.nodes(data=True):
        node_data[1]['node'].threshold = threshold
    ani = animate_network(network, steps=50)
    return HTML(ani.to_jshtml())

# Link sliders to the function
interact(update_network, nodes=node_slider, threshold=threshold_slider)


ani = animate_network(network, steps=50)
HTML(ani.to_jshtml())
